<a href="https://colab.research.google.com/github/chernoskutov1989dv/-Keras-Tuner-/blob/main/_keras_tuner_fashion_mnist_test_version_repeat_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Установка Keras Tuner

In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 38.5 MB/s 
     |████████████████████████████████| 1.6 MB 46.8 MB/s 


## Подключаем нужные пакеты

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files 
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


<ipython-input-2-a3bcebe33b05>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization


In [ ]:
import keras_tuner

## Подготовка данных для обучения сети

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [ ]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                  min_value=512,    # минимальное количество нейронов - 128
                                  max_value=1024,   # максимальное количество - 1024
                                  step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [ ]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )



## Запускаем подбор гиперпараметров

Пространство поиска

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


Подбор гиперпараметров

In [ ]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 80 Complete [00h 00m 17s]
val_accuracy: 0.7302500009536743

Best val_accuracy So Far: 0.9004999995231628
Total elapsed time: 00h 25m 58s


## Выбираем лучшую модель

In [ ]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation: relu
units_input: 640
units_hidden: 224
optimizer: adam
Score: 0.9012500047683716
Trial summary
Hyperparameters:
activation: tanh
units_input: 672
units_hidden: 544
optimizer: adam
Score: 0.9006666541099548
Trial summary
Hyperparameters:
activation: tanh
units_input: 960
units_hidden: 512
optimizer: rmsprop
Score: 0.8998333215713501
Trial summary
Hyperparameters:
activation: relu
units_input: 928
units_hidden: 352
optimizer: adam
Score: 0.8989999890327454
Trial summary
Hyperparameters:
activation: relu
units_input: 928
units_hidden: 480
optimizer: adam
Score: 0.8989999890327454
Trial summary
Hyperparameters:
activation: relu
units_input: 608
units_hidden: 320
optimizer: adam
Score: 0.8985833525657654
Trial summary
Hyperparameters:
activation: relu
units_input: 608
units_hidden: 480
optimizer: adam
Score: 0.8981666564941406
Trial summary
Hyperparameters:
activatio

Получаем три лучших модели

In [ ]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [ ]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 640)               502400    
                                                                 
 dense_1 (Dense)             (None, 224)               143584    
                                                                 
 dense_2 (Dense)             (None, 10)                2250      
                                                                 
Total params: 648,234
Trainable params: 648,234
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.3450 - accuracy: 0.8907

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 672)               527520    
      